In [22]:
import findspark
import collections
from pyspark import SparkContext
from pyspark.sql import SparkSession

findspark.init()

spark = SparkSession.builder \
    .appName("JupyterLocalSpark") \
    .master("local[*]") \
    .getOrCreate()

sc: SparkContext = spark.sparkContext

In [23]:
rdd = sc.textFile("data/1800.csv")
rdd.take(5)

['ITE00100554,18000101,TMAX,-75,,,E,',
 'ITE00100554,18000101,TMIN,-148,,,E,',
 'GM000010962,18000101,PRCP,0,,,E,',
 'EZE00100082,18000101,TMAX,-86,,,E,',
 'EZE00100082,18000101,TMIN,-135,,,E,']

In [24]:
def extract_values(line:str):
    line = line.split(",")
    station_id = line[0]
    flag = line[2]
    temp = int(line[3])
    
    return station_id, flag, temp

rdd = rdd.map(extract_values)
rdd.take(5)

[('ITE00100554', 'TMAX', -75),
 ('ITE00100554', 'TMIN', -148),
 ('GM000010962', 'PRCP', 0),
 ('EZE00100082', 'TMAX', -86),
 ('EZE00100082', 'TMIN', -135)]

In [25]:
rdd = rdd.filter(lambda x : x[1]=="TMIN")
rdd.take(5)

[('ITE00100554', 'TMIN', -148),
 ('EZE00100082', 'TMIN', -135),
 ('ITE00100554', 'TMIN', -125),
 ('EZE00100082', 'TMIN', -130),
 ('ITE00100554', 'TMIN', -46)]

In [26]:
rdd = rdd.map(lambda x : (x[0], x[2]))
rdd.take(5)

[('ITE00100554', -148),
 ('EZE00100082', -135),
 ('ITE00100554', -125),
 ('EZE00100082', -130),
 ('ITE00100554', -46)]

In [27]:
rdd = rdd.reduceByKey(lambda x,y : min(x,y))
rdd.take(5)

[('ITE00100554', -148), ('EZE00100082', -135)]